In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
# pip install lxml

## Import Libraries

In [4]:
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import time

In [5]:
base_url = "https://www.nfl.com"

## Player Stats

#### Define function to get links

In [6]:
def get_links(level, base_url="https://www.nfl.com"):
    if level == "player":
        # URL for player-level stats
        url = "https://www.nfl.com/stats/player-stats"
    elif level == "team":
        unit_list = ["offense", "defense", "special-teams"]
        category_links = {}
        stat_cols = {
            "offense": ["passing", "rushing", "receiving", "scoring", "downs"],
            "defense": ["passing", "rushing", "receiving", "scoring", "tackles", "downs", "fumbles", "interceptions"],
            "special-teams": ["field-goals", "scoring", "kickoffs", "kickoff-returns", "punting", "punt-returns"]
        }

        base_team_stats_url = "https://www.nfl.com/stats/team-stats/"

        for unit in unit_list:
            unit_data = {}
            for stat_col in stat_cols[unit]:
                # Construct the URL for each category in stat_cols
                url = f"{base_team_stats_url}{unit}/2023/{stat_col}"
                unit_data[stat_col.capitalize()] = url

            # Add the unit's data to the main dictionary
            category_links[unit.capitalize()] = unit_data
        print("this is in get_links")        
        category_links
        return category_links

    # For all other cases (invalid level), fetch general category links
    html = requests.get(url)
    soup = BeautifulSoup(html.content, "html.parser")
    a_elements = soup.find_all('li', class_='d3-o-tabs__list-item')
    tabs = [element.find('a').get_text() for element in a_elements]
    links = [base_url + element.find('a')['href'] for element in a_elements]
    print(links)

    return dict(zip(tabs, links))


In [7]:
unit_list = ["offense", "defense", "special-teams"]

html = requests.get("https://www.nfl.com/stats/team-stats/")
soup = BeautifulSoup(html.content, "html.parser")
a_elements = soup.find_all('li', class_='d3-o-tabs__list-item')
tabs = [element.find('a').get_text() for element in a_elements]
links = [base_url + element.find('a')['href'] for element in a_elements]
print(links)

['https://www.nfl.com/stats/team-stats/offense/passing/2023/reg/all', 'https://www.nfl.com/stats/team-stats/offense/rushing/2023/reg/all', 'https://www.nfl.com/stats/team-stats/offense/receiving/2023/reg/all', 'https://www.nfl.com/stats/team-stats/offense/scoring/2023/reg/all', 'https://www.nfl.com/stats/team-stats/offense/downs/2023/reg/all']


In [8]:
unit_list = ["offense", "defense", "special-teams"]
category_links = {}
stat_cols = {
    "offense": ["passing", "rushing", "receiving", "scoring", "downs"],
    "defense": ["passing", "rushing", "receiving", "scoring", "tackles", "downs", "fumbles", "interceptions"],
    "special-teams": ["field-goals", "scoring", "kickoffs", "kickoff-returns", "punting", "punt-returns"]
}

base_team_stats_url = "https://www.nfl.com/stats/team-stats/"

for unit in unit_list:
    unit_data = {}
    for stat_col in stat_cols[unit]:
        # Construct the URL for each category in stat_cols
        url = f"{base_team_stats_url}{unit}/2023/{stat_col}"
        unit_data[stat_col.capitalize()] = url

    # Add the unit's data to the main dictionary
    category_links[unit.capitalize()] = unit_data

for unit, categories in category_links.items():
    print(unit)
    for category, link in categories.items():
        print(f"'{category}': '{link}'")
        
category_links


Offense
'Passing': 'https://www.nfl.com/stats/team-stats/offense/2023/passing'
'Rushing': 'https://www.nfl.com/stats/team-stats/offense/2023/rushing'
'Receiving': 'https://www.nfl.com/stats/team-stats/offense/2023/receiving'
'Scoring': 'https://www.nfl.com/stats/team-stats/offense/2023/scoring'
'Downs': 'https://www.nfl.com/stats/team-stats/offense/2023/downs'
Defense
'Passing': 'https://www.nfl.com/stats/team-stats/defense/2023/passing'
'Rushing': 'https://www.nfl.com/stats/team-stats/defense/2023/rushing'
'Receiving': 'https://www.nfl.com/stats/team-stats/defense/2023/receiving'
'Scoring': 'https://www.nfl.com/stats/team-stats/defense/2023/scoring'
'Tackles': 'https://www.nfl.com/stats/team-stats/defense/2023/tackles'
'Downs': 'https://www.nfl.com/stats/team-stats/defense/2023/downs'
'Fumbles': 'https://www.nfl.com/stats/team-stats/defense/2023/fumbles'
'Interceptions': 'https://www.nfl.com/stats/team-stats/defense/2023/interceptions'
Special-teams
'Field-goals': 'https://www.nfl.com

{'Offense': {'Passing': 'https://www.nfl.com/stats/team-stats/offense/2023/passing',
  'Rushing': 'https://www.nfl.com/stats/team-stats/offense/2023/rushing',
  'Receiving': 'https://www.nfl.com/stats/team-stats/offense/2023/receiving',
  'Scoring': 'https://www.nfl.com/stats/team-stats/offense/2023/scoring',
  'Downs': 'https://www.nfl.com/stats/team-stats/offense/2023/downs'},
 'Defense': {'Passing': 'https://www.nfl.com/stats/team-stats/defense/2023/passing',
  'Rushing': 'https://www.nfl.com/stats/team-stats/defense/2023/rushing',
  'Receiving': 'https://www.nfl.com/stats/team-stats/defense/2023/receiving',
  'Scoring': 'https://www.nfl.com/stats/team-stats/defense/2023/scoring',
  'Tackles': 'https://www.nfl.com/stats/team-stats/defense/2023/tackles',
  'Downs': 'https://www.nfl.com/stats/team-stats/defense/2023/downs',
  'Fumbles': 'https://www.nfl.com/stats/team-stats/defense/2023/fumbles',
  'Interceptions': 'https://www.nfl.com/stats/team-stats/defense/2023/interceptions'},
 '

#### Define Function to format links

In [9]:
def format_links(level):
    # Create a dictionary to store the links for each unit and its categories
    unit_links = {}
    lst = ["offense", "defense", "special-teams"]

    # Process the URL and build the unit_links dictionary
    if level == "player":
        # Define the URL for the player level
        url = "https://www.nfl.com"
        unit_links[level.capitalize()] = get_links(level, url)  # Pass the URL as an argument

    if level == "team":
        for unit in lst:
            # Initialize an empty dictionary for the current unit
            unit_data = {}
            
            # Define the URL for the current unit
            url = f"https://www.nfl.com/stats/team-stats/{unit.lower()}/2023"
            category_links = get_links(level, url)  # Pass the URL as an argument

            # Assuming category_links is your nested dictionary
            for unit, categories in category_links.items():
                for category, link in categories.items():
                    # Remove '/2023' from the link first
                    link = link.replace('/2023', '')
                    # Append '/2023/reg/all' to the modified link
                    category_links[unit][category] = f"{link}/2023/reg/all"


            print('category links are: ', category_links) 
            # Add the category links to the unit_data dictionary
            #unit_data.update(category_links)
            
            # Add the unit_data dictionary to the unit_links dictionary
            unit_links = category_links
            #unit_links[unit.capitalize()] = unit_data

    return unit_links


In [10]:
### Scrape and export to DataFrame

### Set parameters

### Define Function to Check/Create Directory

In [11]:
# Function to create directories if they don't exist
def create_directory_if_not_exists(directory_path):
    if not os.path.exists(directory_path):
        try:
            os.makedirs(directory_path)
            print(f'Directory "{directory_path}" has been created.')
        except OSError as e:
            print(f'Error: Failed to create directory "{directory_path}".')
            print(e)
    else:
        print(f'Directory "{directory_path}" already exists.')


### Define Function to Scrape and Process Data

In [12]:
# Function to scrape and process data
def scrape_and_process_data(link, unit, category, level, unit_directory_path):
    # Request raw HTML for the current page
    response = requests.get(link)

    # Check if the request was successful
    if response.status_code == 200:
        # Create a BeautifulSoup object to parse the HTML
        soup = BeautifulSoup(response.content, "html.parser")

        # Find all elements with the class 'd3-o-player-stats--detailed'
        stats = soup.find_all(attrs={"class": f'd3-o-{level}-stats--detailed'})

        # Initialize lists to collect data
        stat_val = []
        stat_col = []

        # Loop through each <tr> element to extract and collect the text from <td> elements
        for row in stats:
            # This gets the stat names
            header_cells = row.find_all('th')

            if len(header_cells) > 0:
                for cell in header_cells:
                    stat_col.append(cell.get_text(strip=True))

            # This gets the stats
            data_cells = row.find_all('td')

            if len(data_cells) > 0:
                for cell in data_cells:
                    stat_val.append(cell.get_text(strip=True))

        # Determine the number of columns in each row
        num_columns = len(stat_col) if stat_col else 1  # Use 1 if stat_col is empty
        
        # Split the list into rows
        rows = [stat_val[i:i + num_columns] for i in range(0, len(stat_val), num_columns)]

        # Create a DataFrame for the current category
        df = pd.DataFrame(rows, columns=stat_col)
        # Convert all columns except "Team" to numeric
        #numeric_columns = df.columns.difference(['Team'])
        #df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')
        print(df)
        # Check if the DataFrame has a "Team" column before attempting to remove the duplicated part
        if 'Team' in df.columns:
            # Remove duplicated part from the "Team" column
            df['Team'] = df['Team'].apply(lambda x: x[:len(x)//2])

        # Specify the file path within the unit's directory
        csv_file_path = os.path.join(unit_directory_path, category + '.csv')

        # Export the DataFrame to a CSV file
        df.to_csv(csv_file_path, index=False)  # Set index=False to exclude the index column

        print(f'DataFrame for category "{category}" in unit "{unit}" has been exported to {csv_file_path}')
    else:
        print(f"Error: Unable to fetch the page for category '{category}' in unit '{unit}'.")

#### Initiate Scraping Process

In [13]:
def get_stats(level):
    lst = ["offense", "defense", "special-teams"]
    # Define the unit links based on the specified level
    unit_links = format_links(level)
    print("unit links are:")
    print(unit_links)
    # Combine the base directory path with the current week
    directory_path = os.path.join('data', level, 'week' + str(current_week))
    print("lul")
    print(category_links)
    

    if level == "team":
        for unit, categories in category_links.items():
            for category, link in categories.items():
                print('unit is: ', unit)
                print('category is: ', category)
                print('link is: ', link)

                # Create a subdirectory for the current unit
                unit_directory_path = os.path.join(directory_path, unit)
                create_directory_if_not_exists(unit_directory_path)

                # Call scrape_and_process_data with individual category URLs
                scrape_and_process_data(link, unit, category, level, unit_directory_path)
                
    elif level == "player":
        # Directly use the week1 directory for player-level data
        unit_directory_path = directory_path

        # Call scrape_and_process_data with the unit_links dictionary

        for category, link in unit_links.items():
            scrape_and_process_data(link, level, category, level, unit_directory_path)
    else:
        print("Invalid level specified.")


In [14]:
# Define the URL to scrape based on the specified level
#url = f"{base_url}/stats/{level}-stats"
#print(url)

In [15]:
level = "team"  # Replace with "player" or "team" as needed
current_week =  1
get_stats(level)

this is in get_links
category links are:  {'Offense': {'Passing': 'https://www.nfl.com/stats/team-stats/offense/passing/2023/reg/all', 'Rushing': 'https://www.nfl.com/stats/team-stats/offense/rushing/2023/reg/all', 'Receiving': 'https://www.nfl.com/stats/team-stats/offense/receiving/2023/reg/all', 'Scoring': 'https://www.nfl.com/stats/team-stats/offense/scoring/2023/reg/all', 'Downs': 'https://www.nfl.com/stats/team-stats/offense/downs/2023/reg/all'}, 'Defense': {'Passing': 'https://www.nfl.com/stats/team-stats/defense/passing/2023/reg/all', 'Rushing': 'https://www.nfl.com/stats/team-stats/defense/rushing/2023/reg/all', 'Receiving': 'https://www.nfl.com/stats/team-stats/defense/receiving/2023/reg/all', 'Scoring': 'https://www.nfl.com/stats/team-stats/defense/scoring/2023/reg/all', 'Tackles': 'https://www.nfl.com/stats/team-stats/defense/tackles/2023/reg/all', 'Downs': 'https://www.nfl.com/stats/team-stats/defense/downs/2023/reg/all', 'Fumbles': 'https://www.nfl.com/stats/team-stats/def

Error: Unable to fetch the page for category 'Passing' in unit 'Offense'.
unit is:  Offense
category is:  Rushing
link is:  https://www.nfl.com/stats/team-stats/offense/2023/rushing
Directory "data\team\week1\Offense" already exists.
Error: Unable to fetch the page for category 'Rushing' in unit 'Offense'.
unit is:  Offense
category is:  Receiving
link is:  https://www.nfl.com/stats/team-stats/offense/2023/receiving
Directory "data\team\week1\Offense" already exists.
Error: Unable to fetch the page for category 'Receiving' in unit 'Offense'.
unit is:  Offense
category is:  Scoring
link is:  https://www.nfl.com/stats/team-stats/offense/2023/scoring
Directory "data\team\week1\Offense" already exists.
Error: Unable to fetch the page for category 'Scoring' in unit 'Offense'.
unit is:  Offense
category is:  Downs
link is:  https://www.nfl.com/stats/team-stats/offense/2023/downs
Directory "data\team\week1\Offense" already exists.
Error: Unable to fetch the page for category 'Downs' in unit '

In [16]:
# Example usage:
level = "player"  # Replace with "team" or "player" as needed
get_stats(level)



['https://www.nfl.com/stats/player-stats/category/passing/2023/reg/all/passingyards/desc', 'https://www.nfl.com/stats/player-stats/category/rushing/2023/reg/all/rushingyards/desc', 'https://www.nfl.com/stats/player-stats/category/receiving/2023/reg/all/receivingreceptions/desc', 'https://www.nfl.com/stats/player-stats/category/fumbles/2023/reg/all/defensiveforcedfumble/desc', 'https://www.nfl.com/stats/player-stats/category/tackles/2023/reg/all/defensivecombinetackles/desc', 'https://www.nfl.com/stats/player-stats/category/interceptions/2023/reg/all/defensiveinterceptions/desc', 'https://www.nfl.com/stats/player-stats/category/field-goals/2023/reg/all/kickingfgmade/desc', 'https://www.nfl.com/stats/player-stats/category/kickoffs/2023/reg/all/kickofftotal/desc', 'https://www.nfl.com/stats/player-stats/category/kickoff-returns/2023/reg/all/kickreturnsaverageyards/desc', 'https://www.nfl.com/stats/player-stats/category/punts/2023/reg/all/puntingaverageyards/desc', 'https://www.nfl.com/sta

InvalidSchema: No connection adapters were found for "{'Passing': 'https://www.nfl.com/stats/player-stats/category/passing/2023/reg/all/passingyards/desc', 'Rushing': 'https://www.nfl.com/stats/player-stats/category/rushing/2023/reg/all/rushingyards/desc', 'Receiving': 'https://www.nfl.com/stats/player-stats/category/receiving/2023/reg/all/receivingreceptions/desc', 'Fumbles': 'https://www.nfl.com/stats/player-stats/category/fumbles/2023/reg/all/defensiveforcedfumble/desc', 'Tackles': 'https://www.nfl.com/stats/player-stats/category/tackles/2023/reg/all/defensivecombinetackles/desc', 'Interceptions': 'https://www.nfl.com/stats/player-stats/category/interceptions/2023/reg/all/defensiveinterceptions/desc', 'Field Goals': 'https://www.nfl.com/stats/player-stats/category/field-goals/2023/reg/all/kickingfgmade/desc', 'Kickoffs': 'https://www.nfl.com/stats/player-stats/category/kickoffs/2023/reg/all/kickofftotal/desc', 'Kickoff Returns': 'https://www.nfl.com/stats/player-stats/category/kicko

In [ ]:
#d3-o-tabbed-controls-selector__list


# Request raw HTML
html = requests.get("https://www.nfl.com/stats/team-stats/")


# Assuming your HTML is stored in a variable named html_content
soup = BeautifulSoup(html.content, 'html.parser')

# Find the ul element with class 'd3-o-tabbed-controls-selector__list'
ul_element = soup.find('ul', class_='d3-o-tabbed-controls-selector__list')

# Find all li elements within the ul element
li_elements = ul_element.find_all('li')

# Initialize a list to store the href values
href_values = []

# Iterate through the li elements and extract the href values from the a tags
for li in li_elements:
    a_tag = li.find('a')
    if a_tag:
        href = a_tag['href']
        href_values.append(href)


print(href_values)



In [ ]:
import requests
from bs4 import BeautifulSoup

# Sample href_values
href_values = [
    "/stats/team-stats/offense/passing/2023/reg/all",
    "/stats/team-stats/defense/passing/2023/reg/all",
    "/stats/team-stats/special-teams/scoring/2023/reg/all"
]

# Initialize the result dictionary
result_dict = {}

# Iterate through href_values
for href in href_values:
    # Split the href into parts using "/"
    parts = href.split("/")
    
    # Find the key and subkey
    key = parts[3]  # e.g., "offense", "defense", "special-teams"
    subkey = parts[4]  # e.g., "passing", "scoring"
    
    # Create the key if it doesn't exist
    if key not in result_dict:
        result_dict[key] = {}
    
    # Add the subkey and full link to the sub-dictionary
    result_dict[key][subkey] = f"https://www.nfl.com{href}"

# Print the resulting dictionary
print(result_dict)


In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# pip install lxml

## Import Libraries

In [ ]:
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import time

In [ ]:
base_url = "https://www.nfl.com"

## Player Stats

#### Define function to get links

In [ ]:
def get_links(level, base_url="https://www.nfl.com"):
    if level == "player":
        # URL for player-level stats
        url = "https://www.nfl.com/stats/player-stats"
    elif level == "team":
        unit_list = ["offense", "defense", "special-teams"]
        category_links = {}
        stat_cols = {
            "offense": ["passing", "rushing", "receiving", "scoring", "downs"],
            "defense": ["passing", "rushing", "receiving", "scoring", "tackles", "downs", "fumbles", "interceptions"],
            "special-teams": ["field-goals", "scoring", "kickoffs", "kickoff-returns", "punting", "punt-returns"]
        }

        base_team_stats_url = "https://www.nfl.com/stats/team-stats/"

        for unit in unit_list:
            unit_data = {}
            for stat_col in stat_cols[unit]:
                # Construct the URL for each category in stat_cols
                url = f"{base_team_stats_url}{unit}/2023/{stat_col}"
                unit_data[stat_col.capitalize()] = url

            # Add the unit's data to the main dictionary
            category_links[unit.capitalize()] = unit_data
        print("this is in get_links")        
        category_links
        return category_links

    # For all other cases (invalid level), fetch general category links
    html = requests.get(url)
    soup = BeautifulSoup(html.content, "html.parser")
    a_elements = soup.find_all('li', class_='d3-o-tabs__list-item')
    tabs = [element.find('a').get_text() for element in a_elements]
    links = [base_url + element.find('a')['href'] for element in a_elements]
    print(links)

    return dict(zip(tabs, links))


In [ ]:
unit_list = ["offense", "defense", "special-teams"]
category_links = {}
stat_cols = {
    "offense": ["passing", "rushing", "receiving", "scoring", "downs"],
    "defense": ["passing", "rushing", "receiving", "scoring", "tackles", "downs", "fumbles", "interceptions"],
    "special-teams": ["field-goals", "scoring", "kickoffs", "kickoff-returns", "punting", "punt-returns"]
}

base_team_stats_url = "https://www.nfl.com/stats/team-stats/"

for unit in unit_list:
    unit_data = {}
    for stat_col in stat_cols[unit]:
        # Construct the URL for each category in stat_cols
        url = f"{base_team_stats_url}{unit}/2023/{stat_col}"
        unit_data[stat_col.capitalize()] = url

    # Add the unit's data to the main dictionary
    category_links[unit.capitalize()] = unit_data

for unit, categories in category_links.items():
    print(unit)
    for category, link in categories.items():
        print(f"'{category}': '{link}'")
        
category_links


#### Define Function to format links

In [ ]:
def format_links(level):
    # Create a dictionary to store the links for each unit and its categories
    unit_links = {}
    lst = ["offense", "defense", "special-teams"]

    # Process the URL and build the unit_links dictionary
    if level == "player":
        # Define the URL for the player level
        url = "https://www.nfl.com"
        unit_links[level.capitalize()] = get_links(level, url)  # Pass the URL as an argument

    if level == "team":
        for unit in lst:
            # Initialize an empty dictionary for the current unit
            unit_data = {}
            
            # Define the URL for the current unit
            url = f"https://www.nfl.com/stats/team-stats/{unit.lower()}/2023"
            category_links = get_links(level, url)  # Pass the URL as an argument

            # Assuming category_links is your nested dictionary
            for unit, categories in category_links.items():
                for category, link in categories.items():
                    # Remove '/2023' from the link first
                    link = link.replace('/2023', '')
                    # Append '/2023/reg/all' to the modified link
                    category_links[unit][category] = f"{link}/2023/reg/all"


            print('category links are: ', category_links) 
            # Add the category links to the unit_data dictionary
            #unit_data.update(category_links)
            
            # Add the unit_data dictionary to the unit_links dictionary
            unit_links = category_links
            #unit_links[unit.capitalize()] = unit_data

    return unit_links


In [ ]:
### Scrape and export to DataFrame

### Set parameters

### Define Function to Check/Create Directory

In [ ]:
# Function to create directories if they don't exist
def create_directory_if_not_exists(directory_path):
    if not os.path.exists(directory_path):
        try:
            os.makedirs(directory_path)
            print(f'Directory "{directory_path}" has been created.')
        except OSError as e:
            print(f'Error: Failed to create directory "{directory_path}".')
            print(e)
    else:
        print(f'Directory "{directory_path}" already exists.')


### Define Function to Scrape and Process Data

In [ ]:
# Function to scrape and process data
def scrape_and_process_data(link, unit, category, level, unit_directory_path):
    # Request raw HTML for the current page
    response = requests.get(link)

    # Check if the request was successful
    if response.status_code == 200:
        # Create a BeautifulSoup object to parse the HTML
        soup = BeautifulSoup(response.content, "html.parser")

        # Find all elements with the class 'd3-o-player-stats--detailed'
        stats = soup.find_all(attrs={"class": f'd3-o-{level}-stats--detailed'})

        # Initialize lists to collect data
        stat_val = []
        stat_col = []

        # Loop through each <tr> element to extract and collect the text from <td> elements
        for row in stats:
            # This gets the stat names
            header_cells = row.find_all('th')

            if len(header_cells) > 0:
                for cell in header_cells:
                    stat_col.append(cell.get_text(strip=True))

            # This gets the stats
            data_cells = row.find_all('td')

            if len(data_cells) > 0:
                for cell in data_cells:
                    stat_val.append(cell.get_text(strip=True))

        # Determine the number of columns in each row
        num_columns = len(stat_col) if stat_col else 1  # Use 1 if stat_col is empty
        
        # Split the list into rows
        rows = [stat_val[i:i + num_columns] for i in range(0, len(stat_val), num_columns)]

        # Create a DataFrame for the current category
        df = pd.DataFrame(rows, columns=stat_col)
        # Convert all columns except "Team" to numeric
        #numeric_columns = df.columns.difference(['Team'])
        #df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')
        print(df)
        # Check if the DataFrame has a "Team" column before attempting to remove the duplicated part
        if 'Team' in df.columns:
            # Remove duplicated part from the "Team" column
            df['Team'] = df['Team'].apply(lambda x: x[:len(x)//2])

        # Specify the file path within the unit's directory
        csv_file_path = os.path.join(unit_directory_path, category + '.csv')

        # Export the DataFrame to a CSV file
        df.to_csv(csv_file_path, index=False)  # Set index=False to exclude the index column

        print(f'DataFrame for category "{category}" in unit "{unit}" has been exported to {csv_file_path}')
    else:
        print(f"Error: Unable to fetch the page for category '{category}' in unit '{unit}'.")

#### Initiate Scraping Process

In [22]:
def get_stats(level):
    lst = ["offense", "defense", "special-teams"]
    # Define the unit links based on the specified level

    # Combine the base directory path with the current week
    directory_path = os.path.join('data', level, 'week' + str(current_week))
    unit_links = format_links(level)

    print("unit links are:")
    print(key_value_list)
    """
    unit_links = format_links(level)


    print("lul")
    print(category_links)
    """
    

    if level == "team":
        for unit, categories in team_stats_dict.items(): ## change
            for category, link in categories.items():
                print('unit is: ', unit)
                print('category is: ', category)
                print('link is: ', link)

                # Create a subdirectory for the current unit
                unit_directory_path = os.path.join(directory_path, unit)
                create_directory_if_not_exists(unit_directory_path)

                # Call scrape_and_process_data with individual category URLs
                scrape_and_process_data(link, unit, category, level, unit_directory_path)
                
    elif level == "player":
        # Directly use the week1 directory for player-level data
        unit_directory_path = directory_path

        # Call scrape_and_process_data with the unit_links dictionary

        for category, link in unit_links.items():
            scrape_and_process_data(link, level, category, level, unit_directory_path)
    else:
        print("Invalid level specified.")


In [ ]:
# Define the URL to scrape based on the specified level
#url = f"{base_url}/stats/{level}-stats"
#print(url)

In [23]:
# Example usage:
level = "player"  # Replace with "team" or "player" as needed
get_stats(level)



['https://www.nfl.com/stats/player-stats/category/passing/2023/reg/all/passingyards/desc', 'https://www.nfl.com/stats/player-stats/category/rushing/2023/reg/all/rushingyards/desc', 'https://www.nfl.com/stats/player-stats/category/receiving/2023/reg/all/receivingreceptions/desc', 'https://www.nfl.com/stats/player-stats/category/fumbles/2023/reg/all/defensiveforcedfumble/desc', 'https://www.nfl.com/stats/player-stats/category/tackles/2023/reg/all/defensivecombinetackles/desc', 'https://www.nfl.com/stats/player-stats/category/interceptions/2023/reg/all/defensiveinterceptions/desc', 'https://www.nfl.com/stats/player-stats/category/field-goals/2023/reg/all/kickingfgmade/desc', 'https://www.nfl.com/stats/player-stats/category/kickoffs/2023/reg/all/kickofftotal/desc', 'https://www.nfl.com/stats/player-stats/category/kickoff-returns/2023/reg/all/kickreturnsaverageyards/desc', 'https://www.nfl.com/stats/player-stats/category/punts/2023/reg/all/puntingaverageyards/desc', 'https://www.nfl.com/sta

NameError: name 'nested_dict' is not defined

In [ ]:
level = "team"  # Replace with "player" or "team" as needed
    ent_week =  1
get_stats(level)

unit is:  offense
category is:  passing
link is:  https://www.nfl.com/stats/team-stats/offense/passing/2023/reg/all
Directory "data\team\week1\offense" already exists.
                    Team Att Cmp Cmp % Yds/Att Pass Yds TD INT   Rate 1st  \
0       PatriotsPatriots  54  35  64.8     5.8      316  3   1   91.3  18   
1       SteelersSteelers  46  31  67.4       5      232  1   2   68.4   9   
2       DolphinsDolphins  45  28  62.2    10.4      466  3   1    110  23   
3           TexansTexans  44  28  63.6     5.5      242  0   0     78  13   
4         VikingsVikings  44  33    75     7.8      344  2   1  102.8  15   
5             BillsBills  41  29  70.7     5.8      236  1   3   62.6  11   
6             ColtsColts  39  24  61.5     5.7      223  1   1     75  12   
7           ChiefsChiefs  39  21  53.8     5.8      226  2   1   77.5  10   
8       PanthersPanthers  38  20  52.6     3.8      146  1   2   48.8   9   
9               RamsRams  38  24  63.2     8.8      334  0   0

In [ ]:
#d3-o-tabbed-controls-selector__list


# Request raw HTML
html = requests.get("https://www.nfl.com/stats/team-stats/")


# Assuming your HTML is stored in a variable named html_content
soup = BeautifulSoup(html.content, 'html.parser')

# Find the ul element with class 'd3-o-tabbed-controls-selector__list'
ul_element = soup.find('ul', class_='d3-o-tabbed-controls-selector__list')

# Find all li elements within the ul element
li_elements = ul_element.find_all('li')

# Initialize a list to store the href values
href_values = []

# Iterate through the li elements and extract the href values from the a tags
for li in li_elements:
    a_tag = li.find('a')
    if a_tag:
        href = a_tag['href']
        href_values.append(href)


print(href_values)



['/stats/team-stats/offense/passing/2023/reg/all', '/stats/team-stats/defense/passing/2023/reg/all', '/stats/team-stats/special-teams/scoring/2023/reg/all']


In [ ]:
import requests
from bs4 import BeautifulSoup

# Assuming href_values contains the extracted href values as mentioned earlier

base_url = "https://www.nfl.com"  # Replace with your base URL

# Initialize a list to store the links
all_links = []

# Loop through href_values and fetch links for each URL
for href in href_values:
    url = base_url + href
    html = requests.get(url)
    soup = BeautifulSoup(html.content, "html.parser")
    a_elements = soup.find_all('li', class_='d3-o-tabs__list-item')
    links = [base_url + element.find('a')['href'] for element in a_elements]
    all_links.extend(links)  # Append the links to the all_links list

# Print all the links
for link in all_links:
    print(link)

all_links

https://www.nfl.com/stats/team-stats/offense/passing/2023/reg/all
https://www.nfl.com/stats/team-stats/offense/rushing/2023/reg/all
https://www.nfl.com/stats/team-stats/offense/receiving/2023/reg/all
https://www.nfl.com/stats/team-stats/offense/scoring/2023/reg/all
https://www.nfl.com/stats/team-stats/offense/downs/2023/reg/all
https://www.nfl.com/stats/team-stats/defense/passing/2023/reg/all
https://www.nfl.com/stats/team-stats/defense/rushing/2023/reg/all
https://www.nfl.com/stats/team-stats/defense/receiving/2023/reg/all
https://www.nfl.com/stats/team-stats/defense/scoring/2023/reg/all
https://www.nfl.com/stats/team-stats/defense/tackles/2023/reg/all
https://www.nfl.com/stats/team-stats/defense/downs/2023/reg/all
https://www.nfl.com/stats/team-stats/defense/fumbles/2023/reg/all
https://www.nfl.com/stats/team-stats/defense/interceptions/2023/reg/all
https://www.nfl.com/stats/team-stats/special-teams/field-goals/2023/reg/all
https://www.nfl.com/stats/team-stats/special-teams/scoring/2

['https://www.nfl.com/stats/team-stats/offense/passing/2023/reg/all',
 'https://www.nfl.com/stats/team-stats/offense/rushing/2023/reg/all',
 'https://www.nfl.com/stats/team-stats/offense/receiving/2023/reg/all',
 'https://www.nfl.com/stats/team-stats/offense/scoring/2023/reg/all',
 'https://www.nfl.com/stats/team-stats/offense/downs/2023/reg/all',
 'https://www.nfl.com/stats/team-stats/defense/passing/2023/reg/all',
 'https://www.nfl.com/stats/team-stats/defense/rushing/2023/reg/all',
 'https://www.nfl.com/stats/team-stats/defense/receiving/2023/reg/all',
 'https://www.nfl.com/stats/team-stats/defense/scoring/2023/reg/all',
 'https://www.nfl.com/stats/team-stats/defense/tackles/2023/reg/all',
 'https://www.nfl.com/stats/team-stats/defense/downs/2023/reg/all',
 'https://www.nfl.com/stats/team-stats/defense/fumbles/2023/reg/all',
 'https://www.nfl.com/stats/team-stats/defense/interceptions/2023/reg/all',
 'https://www.nfl.com/stats/team-stats/special-teams/field-goals/2023/reg/all',
 'ht

In [ ]:
# Create a dictionary to store the links
team_stats_dict = {}

for link in all_links:
    # Split the link by "/"
    parts = link.split('/')
    # Get the keys and values
    unit = parts[5] # e.g., offense, defense, special-teams
    category = parts[6] # e.g., passing, rushing etc.
    year = parts[7] # 2023
    leg = parts[8] # reg of playoffs
    url = link
    # Add to the dictionary
    if unit not in team_stats_dict:
        team_stats_dict[unit] = {}
    team_stats_dict[unit][category] = url
team_stats_dict


{'offense': {'passing': 'https://www.nfl.com/stats/team-stats/offense/passing/2023/reg/all',
  'rushing': 'https://www.nfl.com/stats/team-stats/offense/rushing/2023/reg/all',
  'receiving': 'https://www.nfl.com/stats/team-stats/offense/receiving/2023/reg/all',
  'scoring': 'https://www.nfl.com/stats/team-stats/offense/scoring/2023/reg/all',
  'downs': 'https://www.nfl.com/stats/team-stats/offense/downs/2023/reg/all'},
 'defense': {'passing': 'https://www.nfl.com/stats/team-stats/defense/passing/2023/reg/all',
  'rushing': 'https://www.nfl.com/stats/team-stats/defense/rushing/2023/reg/all',
  'receiving': 'https://www.nfl.com/stats/team-stats/defense/receiving/2023/reg/all',
  'scoring': 'https://www.nfl.com/stats/team-stats/defense/scoring/2023/reg/all',
  'tackles': 'https://www.nfl.com/stats/team-stats/defense/tackles/2023/reg/all',
  'downs': 'https://www.nfl.com/stats/team-stats/defense/downs/2023/reg/all',
  'fumbles': 'https://www.nfl.com/stats/team-stats/defense/fumbles/2023/reg